In [ ]:
# bob.refresh {"cron": "5,30 2-12 * * *", "timeout": 40000}

In [ ]:
%run __init__.ipynb

In [ ]:
BHC = bob.HealthCheck()
BHC_KEY = "51c8f3e8-3d84-4888-878e-a53f0a152e1a"
if USE_HEALTHCHECKS:
    STEP_1 = BHC.start(BHC_KEY)
    print(STEP_1)

In [ ]:
# --- Check if the data are already updated
FILEPATH = f'{OUTPUT_FOLDER}WORLD_DB_TREND.csv'
if os.path.exists(FILEPATH):
    DF_CHECK = pd.read_csv(FILEPATH, sep=';', low_memory=False)
    DATE_CHECK = DF_CHECK[DF_CHECK['SCENARIO'] == YESTERDAY.
                          strftime('%d/%m/%Y')]
    if not DATE_CHECK:
        %run "covid-19_world.ipynb"
    else:
        if USE_HEALTHCHECKS:
            STEP_2 = BHC.done(BHC_KEY)
            print(STEP_2)
            raise Exception(f'Data already updated ! DateTime : {NOW}')
else:
    %run "covid-19_world.ipynb"

In [ ]:
# --- Load data source
START_TIME = time.time()
DB_CONSO = get_datasource('WORLD_DB_CONSO')
DB_ALL = get_datasource('WORLD_DB_ALL')
DB_TREND = get_datasource('WORLD_DB_TREND')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")

In [ ]:
# -- Date selector : ENTITIES
def get_entities(dataf):
    """
    doc string
    """
    cols_to_rename = {"ENTITY": "ENTITIES", "ENTITY_GROUPS": "GROUPS"}
    filters = ['WORLDWIDE', 'Europe', 'Asia', 'Africa', 'North America',
               'South America', 'Oceania', 'Other', 'To be affected']
    dataf = (dataf[dataf['ENTITY_GROUPS'].isin(filters)].
             rename(index=str, columns=cols_to_rename)[['ENTITIES', 'GROUPS']].
             drop_duplicates().
             sort_values(by="ENTITIES", ascending=True).
             reset_index(drop=True))
    dataf['ORDER'] = dataf.index + 8
    dataf.loc[dataf['ENTITIES'] == 'WORLDWIDE', 'ORDER'] = 0
    dataf.loc[dataf['ENTITIES'] == 'Europe', 'ORDER'] = 1
    dataf.loc[dataf['ENTITIES'] == 'Asia', 'ORDER'] = 2
    dataf.loc[dataf['ENTITIES'] == 'Africa', 'ORDER'] = 3
    dataf.loc[dataf['ENTITIES'] == 'North America', 'ORDER'] = 4
    dataf.loc[dataf['ENTITIES'] == 'South America', 'ORDER'] = 5
    dataf.loc[dataf['ENTITIES'] == 'Oceania', 'ORDER'] = 6
    dataf.loc[dataf['ENTITIES'] == 'Other', 'ORDER'] = 7
    dataf.loc[dataf['ENTITIES'] == 'To be affected', 'ORDER'] = 8

    dataf.loc[dataf['ENTITIES'] == 'WORLDWIDE', 'ORDER_GROUPS'] = 0
    dataf.loc[dataf['ENTITIES'] == 'Europe', 'ORDER_GROUPS'] = 1
    dataf.loc[dataf['ENTITIES'] == 'Asia', 'ORDER_GROUPS'] = 2
    dataf.loc[dataf['ENTITIES'] == 'Africa', 'ORDER_GROUPS'] = 3
    dataf.loc[dataf['ENTITIES'] == 'North America', 'ORDER_GROUPS'] = 4
    dataf.loc[dataf['ENTITIES'] == 'South America', 'ORDER_GROUPS'] = 5
    dataf.loc[dataf['ENTITIES'] == 'Oceania', 'ORDER_GROUPS'] = 6
    dataf.loc[dataf['ENTITIES'] == 'Other', 'ORDER_GROUPS'] = 7
    dataf.loc[dataf['ENTITIES'] == 'To be affected', 'ORDER_GROUPS'] = 8

    dataf.loc[dataf['GROUPS'] == 'Europe', 'ORDER_GROUPS'] = 1
    dataf.loc[dataf['GROUPS'] == 'Asia', 'ORDER_GROUPS'] = 2
    dataf.loc[dataf['GROUPS'] == 'Africa', 'ORDER_GROUPS'] = 3
    dataf.loc[dataf['GROUPS'] == 'North America', 'ORDER_GROUPS'] = 4
    dataf.loc[dataf['GROUPS'] == 'South America', 'ORDER_GROUPS'] = 5
    dataf.loc[dataf['GROUPS'] == 'Oceania', 'ORDER_GROUPS'] = 6
    dataf.loc[dataf['GROUPS'] == 'Other', 'ORDER_GROUPS'] = 7
    dataf.loc[dataf['GROUPS'] == 'To be affected', 'ORDER_GROUPS'] = 8

    dataf.loc[dataf['GROUPS'] == 'WORLDWIDE',
              'ENTITY_DISPLAY'] = dataf['ENTITIES']
    dataf.loc[dataf['GROUPS'] != 'WORLDWIDE',
              'ENTITY_DISPLAY'] = '--- ' + dataf['ENTITIES']
    return dataf.sort_values(by=['ORDER_GROUPS', 'ORDER'], ascending=True)


ENTITIES = get_entities(DB_CONSO)
ENTITIES.to_csv(os.path.join(OUTPUT_FOLDER, 'ENTITIES.csv'), sep=";")
if USE_MONGO:
    bob.mongo.save_df(ENTITIES, 'ENTITIES', DB_APP, True)
# ENTITIES

In [ ]:
# -- Date selector : SCENARIOS
def get_scenarios(dataf):
    """
    doc string
    """
    dataf = dataf[['SCENARIO', 'DATE_ORDER']].drop_duplicates()
    dataf['SCENARIO_DIPLAY'] = (pd.to_datetime(dataf['SCENARIO'],
                                               format='%d/%m/%Y').
                                dt.strftime('%d %b %Y'))
    dataf['SCENARIO_TODAY'] = (pd.to_datetime(dataf['SCENARIO'],
                                              format='%d/%m/%Y') +
                               timedelta(days=1)).dt.strftime('%d/%m/%Y')
    dataf['SCENARIO_TODAY_DIPLAY'] = (pd.to_datetime(dataf['SCENARIO_TODAY'],
                                                     format='%d/%m/%Y').
                                      dt.strftime('%d %b %Y'))
    return dataf.sort_values(by='DATE_ORDER', ascending=False)


SCENARIOS = get_scenarios(DB_ALL)
SCENARIOS.to_csv(os.path.join(OUTPUT_FOLDER, 'SCENARIOS.csv'), sep=";")
if USE_MONGO:
    bob.mongo.save_df(SCENARIOS, 'SCENARIOS', DB_APP, True)
# SCENARIOS

In [ ]:
# Dashboard value dynamic template
def dashboard_value(dataf, kpi, domain_num):
    """
    doc string
    """
    filters = ['WORLDWIDE', 'Europe', 'Asia', 'Africa', 'North America',
               'South America', 'Oceania', 'Other', 'To be affected']
    domain = dataf[dataf['ENTITY_GROUPS'].isin(filters) &
                   dataf['KPI'].isin([kpi])]
    domain = pd.DataFrame({'ENTITY': domain['ENTITY'],
                           'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
                           'SCENARIO': domain['SCENARIO'],
                           'DATE_ORDER': domain['DATE_ORDER'],
                           'LABEL': domain['KPI'],
                           'VALUE': domain['VALUE'],
                           'VALUE_D-1': domain['VALUE_D-1'],
                           'VAR': domain['VARV'],
                           'VARP': domain['VARP'],
                           'UNIT_VALUE': domain['UNIT_VALUE'],
                           'UNIT_VAR': domain['UNIT_VAR'],
                           'UNIT_VARP': domain['UNIT_VARP'],
                           'PRECISION_VALUE': domain['PRECISION_VALUE'],
                           'PRECISION_VAR': domain['PRECISION_VAR'],
                           'PRECISION_VARP': domain['PRECISION_VARP'],
                           'LAST_UPDATE': NOW})
    domain = domain.sort_values(by=['ENTITY', 'SCENARIO'], ascending=False)
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=";")
    if USE_MONGO:
        bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),
                                ("SCENARIO", DESCENDING)])
    return domain

In [ ]:
# --- DASHBOARD ---
# -- VALUE DYNAMIC KPIS
START_TIME = time.time()
DOMAIN001 = dashboard_value(DB_ALL, 'Confirmed', '001')
DOMAIN002 = dashboard_value(DB_ALL, 'Deaths', '002')
DOMAIN003 = dashboard_value(DB_ALL, 'Recovered', '003')
DOMAIN004 = dashboard_value(DB_ALL, 'Active cases', '004')
DOMAIN005 = dashboard_value(DB_ALL, "Fatality Rate", '005')
DOMAIN006 = dashboard_value(DB_ALL, "Recovery Rate", '006')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")

In [ ]:
# Duplicate to rank countries WORLDWIDE
def get_ww(dataf):
    """
    doc string
    """
    filters = ['Europe', 'Asia', 'Africa', 'North America',
               'South America', 'Oceania', 'Other', 'To be affected']
    df_ww = dataf.copy()[(dataf['ENTITY'].isin(filters))]
    df_ww['ENTITY'] = "WORLDWIDE"
    dataf = dataf.append(df_ww)
    return dataf

In [ ]:
# Dashboard leaderboard template
def dashboard_leaderboard(dataf, kpi, domain_num):
    """
    doc string
    """
    domain = dataf[(dataf['KPI'].isin([kpi]))]
    domain = pd.DataFrame({'ENTITY': domain['ENTITY_GROUPS'],
                           'SCENARIO': domain['SCENARIO'],
                           'DATE_ORDER': domain['DATE_ORDER'],
                           'LABEL': domain['ENTITY'],
                           'VALUE': domain['VALUE'],
                           'VALUE_D-1': domain['VALUE_D-1'],
                           'VAR': domain['VARV'],
                           'VARP': domain['VARP'],
                           'UNIT_VALUE': domain['UNIT_VALUE'],
                           'UNIT_VAR': domain['UNIT_VAR'],
                           'UNIT_VARP': domain['UNIT_VARP'],
                           'PRECISION_VALUE': domain['PRECISION_VALUE'],
                           'PRECISION_VAR': domain['PRECISION_VAR'],
                           'PRECISION_VARP': domain['PRECISION_VARP'],
                           'LAST_UPDATE': NOW})
    domain = domain.sort_values(by=['ENTITY', 'SCENARIO'], ascending=False)
    if domain_num in ('010', '011', '012'):
        domain = get_ww(domain)
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=";")
    if USE_MONGO:
        bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),
                                ("SCENARIO", DESCENDING)])
    return domain

In [ ]:
# # -- LEADERBOARD BY REGION
# START_TIME = time.time()
# DB_R = DB_ALL[(DB_ALL['ENTITY_GROUPS'] == 'WORLDWIDE') &
#               (DB_ALL['ENTITY'] != 'WORLDWIDE')]
# DOMAIN007 = dashboard_leaderboard(DB_R, 'Confirmed', '007')
# DOMAIN008 = dashboard_leaderboard(DB_R, 'Deaths', '008')
# DOMAIN009 = dashboard_leaderboard(DB_R, 'Recovered', '009')
# print("Script execution completed at "
#       f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
#       f"Time: --- {time.time() - START_TIME} secnds ---")

In [ ]:
# -- LEADERBOARD BY COUNTRIES
START_TIME = time.time()
DB_C = DB_ALL[(DB_ALL['ENTITY_GROUPS'] != 'WORLDWIDE')]
DOMAIN010 = dashboard_leaderboard(DB_C, 'Confirmed', '010')
DOMAIN011 = dashboard_leaderboard(DB_C, 'Deaths', '011')
DOMAIN012 = dashboard_leaderboard(DB_C, 'Recovered', '012')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")

In [ ]:
# -- Covid-19 - TRENDS
START_TIME = time.time()


def domain_101(dataf, domain_num):
    """
    doc string
    """
    # filters = ['Confirmed', 'Deaths', 'Recovered', 'Active cases',
    #            'Deaths ratio', 'Recovered ratio']
    filters = ['Confirmed', 'Deaths', 'Recovered', 'Active cases']
    domain = dataf[dataf['KPI'].isin(filters)]
    domain = pd.DataFrame({'ENTITY': domain['ENTITY'],
                           'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
                           'SCENARIO': domain['SCENARIO'],
                           'DATE_ORDER': domain['DATE_ORDER'],
                           'GROUPS': domain['KPI'],
                           'BOTTOM_FILTER': domain['DATE_SCENARIO'],
                           'UPPER_FILTER_M': domain['METRIC'],
                           'VALUE': domain['VALUE'],
                           'UNIT_VALUE': domain['UNIT'],
                           'PRECISION': domain['PRECISION'],
                           'LAST_UPDATE': NOW})

    # Deconfinement France
    groups_name = 'Deconfinement'
    df_france = domain.copy()[(domain['ENTITY'] == 'France') &
                              (domain['GROUPS'] == 'Confirmed') &
                              (domain['UPPER_FILTER_M'] == 'VARV')]
    df_france.loc[:, 'GROUPS'] = groups_name
    df_france.loc[:, 'VALUE'] = 3000
    domain = pd.concat([domain, df_france], axis=0)

    # Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE',
               'UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') &
               (domain['GROUPS'].isin(['Confirmed', 'Deaths',
                                       'Recovered', 'Active cases',
                                       groups_name])),
               'UPPER_FILTER_M'] = 'DAILY VARIATION (CASES)'
    # domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') &
    #            (domain['GROUPS'].isin(['Deaths ratio', 'Recovered ratio'])),
    #            'UPPER_FILTER_M'] = 'DAILY VARIATION (PTS)'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') &
               (domain['GROUPS'].isin(['Confirmed', 'Deaths',
                                       'Recovered', 'Active cases',
                                       groups_name])),
               'UPPER_FILTER_M'] = 'DAILY VARIATION (%)'
    # domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') &
    #            (domain['GROUPS'].isin(['Deaths ratio',
    #                                    'Recovered ratio'])),
    #            'UPPER_FILTER_M'] = ''

    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=";")
    if USE_MONGO:
        bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),
                                ("UPPER_FILTER_M", DESCENDING),
                                ("BOTTOM_FILTER", DESCENDING)])
    return domain


DOMAIN101 = domain_101(DB_TREND, '101')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
# DOMAIN101

In [ ]:
# -- Covid-19 - TRENDS BY COUNTRIES
START_TIME = time.time()


def domain_102(dataf, domain_num):
    """
    doc string
    """
    domain = dataf.copy()
    max_date = domain['DATE_ORDER'].max()
    filters = ['Europe', 'Asia', 'Africa', 'North America',
               'South America', 'Oceania', 'Other', 'To be affected']
    countries = (domain[(domain['ENTITY_GROUPS'].isin(filters)) &
                        (domain['DATE_ORDER'] == max_date) &
                        (domain['GROUPS'] == 'Confirmed') &
                        (domain['BOTTOM_FILTER'] == 'Since beginning')].
                 sort_values(by='VALUE', ascending=False)['ENTITY'].
                 drop_duplicates().values.tolist()[:10])
    domain = domain[(domain['ENTITY_GROUPS'] != "WORLDWIDE") &
                    (domain['ENTITY'].isin(countries))]

    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=";")
    if USE_MONGO:
        bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("GROUPS", ASCENDING),
                                ("UPPER_FILTER_M", DESCENDING),
                                ("BOTTOM_FILTER", DESCENDING)])
    return domain


DOMAIN102 = domain_102(DOMAIN101, '102')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
# DOMAIN102

In [ ]:
# -- Covid-19 - RANKING BY COUNTRIES
START_TIME = time.time()
# -> Referentials
REF_CONTINENT = pd.read_excel(os.path.join(INPUT_FOLDER, 'REF_WSR.xlsx'),
                              sheet_name='REF_CONTINENT')


def domain_103(dataf, ref, domain_num):
    """
    doc string
    """
    domain = dataf.copy()
    cols_to_rename = {"LABEL": "KPI", "ENTITY": "LABEL",
                      "ENTITY_GROUPS": "ENTITY", "VARV": "VAR"}
    domain = get_ww(domain.drop(['DATE', 'UNIT_VAR',
                                 'UNIT_VARP', 'PRECISION_VALUE',
                                 'PRECISION_VAR', 'PRECISION_VARP'], axis=1).
                    rename(index=str, columns=cols_to_rename).
                    reset_index(drop=True))
    domain = (pd.merge(domain, ref, left_on=['LABEL'],
                       right_on=['COUNTRY_REGION'],
                       how='left').
              drop(['CONTINENT', 'COUNTRY_REGION'], axis=1).
              reset_index(drop=True))

    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=";")
    if USE_MONGO:
        bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),
                                ("SCENARIO", DESCENDING),
                                ("LABEL", DESCENDING),
                                ("KPI", ASCENDING),
                                ("VALUE", DESCENDING)])
    return domain


DOMAIN103 = domain_103(DB_ALL, REF_CONTINENT, '103')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
# DOMAIN103

In [ ]:
# -- Covid-19 - DAILY TREND
START_TIME = time.time()


def domain_106(dataf, domain_num):
    """
    doc string
    """
    # -- Exclude Rates
    dataf = dataf[~dataf['KPI'].isin(['Fatality Rate', "Recovery Rate"])]

    # -- Calculate Value
    filters = ['WORLDWIDE', 'Europe', 'Asia', 'Africa', 'North America',
               'South America', 'Oceania', 'Other', 'To be affected']
    domain_val = dataf.copy()[['ENTITY_GROUPS', 'ENTITY',
                               'SCENARIO', 'KPI',
                               'VALUE', 'VALUE_D-1']][dataf['ENTITY_GROUPS'].
                                                      isin(filters)]
    # [dataf['ENTITY_GROUPS'] == 'WORLDWIDE']
    cols_to_keep = ['ENTITY_GROUPS', 'ENTITY', 'SCENARIO', 'KPI']
    domain_val = domain_val.melt(id_vars=cols_to_keep,
                                 value_name='VALUE',
                                 var_name='METRIC')
    domain_val['ENTITY_GROUPS'] = domain_val['ENTITY']
    domain_val.loc[domain_val['METRIC'] == 'VALUE', 'LABEL_ORDER'] = 99999999
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1', 'LABEL_ORDER'] = 0
    domain_val.loc[domain_val['METRIC'] == 'VALUE',
                   'METRIC'] = domain_val['SCENARIO']
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1',
                   'METRIC'] = (pd.to_datetime(domain_val['SCENARIO'],
                                               format='%d/%m/%Y') +
                                timedelta(days=-1)).dt.strftime('%d/%m/%Y')

    cols_to_rename = {'METRIC': 'LABEL',
                      'ENTITY_GROUPS': 'ENTITY',
                      'KPI': 'UPPER_FILTER_R'}
    domain_val = domain_val.drop('ENTITY',
                                 axis=1).rename(index=str,
                                                columns=cols_to_rename)
    domain_val['PRECISION'] = ',.0f'

    # --Variation
    # filters = ['Europe', 'Asia', 'Africa', 'North America',
    #            'South America', 'Oceania', 'Other', 'To be affected']
    domain_var = dataf.copy()[['ENTITY_GROUPS', 'ENTITY',
                               'SCENARIO', 'KPI',
                               'VARV',
                               'VARP']][dataf['ENTITY_GROUPS'] != 'WORLDWIDE']
    # [df['ENTITY_GROUPS'].isin(filters)]
    cols_to_rename = {'VARV': 'VALUE', 'VARP': 'VAR'}
    domain_var = (domain_var.
                  rename(index=str, columns=cols_to_rename).
                  reset_index(drop=True))
    domain_var = domain_var[domain_var['VALUE'] != 0]

    # -- Get the first 10
    filters = domain_var[['ENTITY_GROUPS',
                          'SCENARIO', 'KPI']].drop_duplicates().values.tolist()
    domain_w = pd.DataFrame()
    for fil in filters:
        tmp_df = (domain_var[(domain_var['ENTITY_GROUPS'] == fil[0]) &
                             (domain_var['SCENARIO'] == fil[1]) &
                             (domain_var['KPI'] == fil[2])].
                  sort_values(by=["VALUE", "ENTITY"], ascending=False).
                  reset_index(drop=True))
        tmp_df['LABEL_ORDER'] = tmp_df.index.astype(int) + 1
        if len(tmp_df) > 9:
            tmp_df.loc[tmp_df.index > 9, 'ENTITY'] = 'Others'
            tmp_df.loc[tmp_df.index > 9, 'LABEL_ORDER'] = 11
            cols_to_group = ['ENTITY_GROUPS', 'ENTITY',
                             'SCENARIO', 'KPI', 'LABEL_ORDER']
            tmp_df = (tmp_df.
                      groupby(cols_to_group, as_index=False).
                      agg({'VALUE': 'sum'}).
                      reset_index(drop=True))
        tmp_df['PRECISION'] = '+,.0f'
        domain_w = domain_w.append(tmp_df)

    cols_to_rename = {'ENTITY': 'LABEL',
                      'ENTITY_GROUPS': 'ENTITY',
                      'KPI': 'UPPER_FILTER_R'}
    domain_w = domain_w.rename(index=str, columns=cols_to_rename)
    domain_w['GROUPS'] = domain_w['LABEL']

    filters = ['Europe', 'Asia', 'Africa', 'North America',
               'South America', 'Oceania', 'Other', 'To be affected']
    cols_to_rename = {'LABEL': 'GROUPS', 'ENTITY': 'LABEL'}
    domain_w2 = (domain_w.
                 copy()[domain_w['ENTITY'].isin(filters)].
                 drop('GROUPS', axis=1).
                 rename(index=str, columns=cols_to_rename))
    domain_w2['ENTITY'] = 'WORLDWIDE'

    domain = pd.concat([domain_val, domain_w, domain_w2], axis=0)

    # -- Save df
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=";")
    if USE_MONGO:
        bob.mongo.save_df(domain, domain_num, DB_APP, True)
    df_collection = MongoDB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),
                                ("SCENARIO", DESCENDING),
                                ("UPPER_FILTER_R", ASCENDING)])
    return domain


DOMAIN106 = domain_106(DB_ALL, '106')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
# DOMAIN106

In [ ]:
if USE_HEALTHCHECKS:
    STEP_2 = BHC.done(BHC_KEY)
    print(STEP_2)